Distribuição normal

In [ ]:
pip install ydata-profiling

In [27]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport
import scipy.stats as stats
import math
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.stats.api as sms

mu = 2
variance = 3
sigma = math.sqrt(variance)
x = np.linspace(mu -3*sigma, mu + 3*sigma, 100)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x,
    y=stats.norm.pdf(x, mu, sigma),
    mode='lines',
    name=' '.join(['Var =', str(variance)])
))
fig.add_trace(go.Scatter(
    x=x,
    y=stats.norm.pdf(x, 0, math.sqrt(1)),
    mode='lines',
    name='Var = 1'
))
fig.show()

Distribuição exponencial

In [ ]:
from scipy.stats import expon
x = np.linspace(expon.ppf(0.01),
                expon.ppf(0.99), 100)
Lambda = 1
fig =go.Figure()
fig.add_trace(go.Scatter(x=x, y=Lambda*expon.pdf(Lambda*x),
                         mode = 'lines',
                         name = ' '))
fig.show()


Distrbuição Gama

In [ ]:
from scipy.stats import gamma

a =1

x = np.linspace(gamma.ppf(0.01, a),
                gamma.ppf(0.99, a), 100)
Lambda = 1
fig =go.Figure()
fig.add_trace(go.Scatter(x=x, y=gamma.pdf(x, a),
                         mode = 'lines',
                         name = ' '))
fig.show()

Correlação

In [ ]:
df = pd.read_csv('/content/sample_data/california_housing_test.csv', sep = ',', decimal = '.')
df.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [ ]:
np.corrcoef(df['housing_median_age'], df['median_income'])

array([[ 1.        , -0.14431531],
       [-0.14431531,  1.        ]])

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1.000000,-0.925017,-0.064203,0.049865,0.070869,0.111572,0.051062,-0.018701,-0.050662
latitude,-0.925017,1.000000,-0.025143,-0.039632,-0.068245,-0.117318,-0.068296,-0.072363,-0.138428
housing_median_age,-0.064203,-0.025143,1.000000,-0.367850,-0.323154,-0.299888,-0.305171,-0.144315,0.091409
total_rooms,0.049865,-0.039632,-0.367850,1.000000,0.937749,0.838867,0.914116,0.221249,0.160427
total_bedrooms,0.070869,-0.068245,-0.323154,0.937749,1.000000,0.856387,0.970758,0.024025,0.082279
population,0.111572,-0.117318,-0.299888,0.838867,0.856387,1.000000,0.895530,0.032361,-0.001192
households,0.051062,-0.068296,-0.305171,0.914116,0.970758,0.895530,1.000000,0.048625,0.100176
median_income,-0.018701,-0.072363,-0.144315,0.221249,0.024025,0.032361,0.048625,1.000000,0.672695
median_house_value,-0.050662,-0.138428,0.091409,0.160427,0.082279,-0.001192,0.100176,0.672695,1.000000


Medidas de tendência

In [ ]:
df['total_bedrooms'].median()


529.9506666666666

In [ ]:
df['total_bedrooms'].mean()

In [ ]:
x = np.sort(df['total_bedrooms'])
x

array([2.000e+00, 3.000e+00, 4.000e+00, ..., 5.027e+03, 5.033e+03,
       5.419e+03])

In [ ]:
df['total_bedrooms'].mode()

In [ ]:
df[df.total_bedrooms == 20].shape[0] # conta quantes vezes o 20 aparece na variável

4

Medidas de Dispersão

In [ ]:
df['total_bedrooms'].max()

In [ ]:
df['total_bedrooms'].min()

In [ ]:
df['total_bedrooms'].var()

172768.55375080582

In [ ]:
df['total_bedrooms'].std()

415.6543681363229

Análise de dados

In [ ]:
stats.shapiro(df['total_bedrooms'])

ShapiroResult(statistic=0.7256292104721069, pvalue=0.0)

In [ ]:
import statsmodels.formula.api as smf
fit = smf.ols('total_bedrooms ~ total_rooms', data=df).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:         total_bedrooms   R-squared:                       0.879
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                 2.186e+04
Date:                Wed, 22 May 2024   Prob (F-statistic):               0.00
Time:                        00:24:10   Log-Likelihood:                -19173.
No. Observations:                3000   AIC:                         3.835e+04
Df Residuals:                    2998   BIC:                         3.836e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      59.8885      4.130     14.500      

Análise de resíduos

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.stats.diagnostic as smd
from statsmodels.compat import lzip

names = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
test = smd.het_breuschpagan(fit.resid, fit.model.exog)
lzip(names, test)


[('Lagrange multiplier statistic', 306.71112845740214),
 ('p-value', 1.136829588789235e-68),
 ('f-value', 341.4115629522619),
 ('f p-value', 2.797583265976525e-72)]

In [ ]:
stats.shapiro(fit.resid)

ShapiroResult(statistic=0.8479374647140503, pvalue=0.0)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['total_bedrooms'], y=df['total_rooms'],
                         mode = 'markers',
                         name = ' '))
fig.show()

In [ ]:
res = fit.get_influence()
y = res.resid_studentized_internal

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(99)), y=y,
                         mode = 'markers',
                         name = ' '))
fig.show()

Teste de hipótese para médias

In [ ]:
stats.ttest_ind(df['total_bedrooms'], df['total_rooms'], equal_var = False)

TtestResult(statistic=-51.636718168162844, pvalue=0.0, df=3221.708864232411)

In [ ]:
# Homogeneidade de variâncias
from scipy.stats import levene
levene(df['total_bedrooms'], df['total_rooms'])

#normalidade
stats.shapiro(df['total_bedrooms'])

ShapiroResult(statistic=0.7256292104721069, pvalue=0.0)

In [ ]:
from scipy.stats.stats import pearsonr
pearsonr(df['total_bedrooms'], df['total_rooms'])

PearsonRResult(statistic=0.9377493550978159, pvalue=0.0)

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1.000000,-0.925017,-0.064203,0.049865,0.070869,0.111572,0.051062,-0.018701,-0.050662
latitude,-0.925017,1.000000,-0.025143,-0.039632,-0.068245,-0.117318,-0.068296,-0.072363,-0.138428
housing_median_age,-0.064203,-0.025143,1.000000,-0.367850,-0.323154,-0.299888,-0.305171,-0.144315,0.091409
total_rooms,0.049865,-0.039632,-0.367850,1.000000,0.937749,0.838867,0.914116,0.221249,0.160427
total_bedrooms,0.070869,-0.068245,-0.323154,0.937749,1.000000,0.856387,0.970758,0.024025,0.082279
population,0.111572,-0.117318,-0.299888,0.838867,0.856387,1.000000,0.895530,0.032361,-0.001192
households,0.051062,-0.068296,-0.305171,0.914116,0.970758,0.895530,1.000000,0.048625,0.100176
median_income,-0.018701,-0.072363,-0.144315,0.221249,0.024025,0.032361,0.048625,1.000000,0.672695
median_house_value,-0.050662,-0.138428,0.091409,0.160427,0.082279,-0.001192,0.100176,0.672695,1.000000


In [14]:
import pandas as pd
from sklearn.datasets import load_iris
from scipy import stats
import numpy as np

iris = pd.read_csv('Iris.csv')

In [ ]:
iris.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,150.0,75.500000,43.445368,1.0,38.25,75.50,112.75,150.0
SepalLengthCm,150.0,5.843333,0.828066,4.3,5.10,5.80,6.40,7.9
SepalWidthCm,150.0,3.054000,0.433594,2.0,2.80,3.00,3.30,4.4
PetalLengthCm,150.0,3.758667,1.764420,1.0,1.60,4.35,5.10,6.9
PetalWidthCm,150.0,1.198667,0.763161,0.1,0.30,1.30,1.80,2.5


In [ ]:
iris['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [ ]:
stats.ttest_ind(iris['SepalLengthCm'], iris['PetalLengthCm'], equal_var = False)

TtestResult(statistic=13.099504494510061, pvalue=4.208973558531982e-29, df=211.59917769493197)

In [ ]:
d1 = iris[iris['Species'] == 'Iris-setosa']
d2 = iris[iris['Species'] == 'Iris-versicolor']
d3 = iris[iris['Species'] == 'Iris-virginica']


In [ ]:
iris['Species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [ ]:
stats.ttest_ind(d1['PetalWidthCm'], d2['PetalWidthCm'], equal_var = False)

TtestResult(statistic=-34.01237858829048, pvalue=1.53779680512802e-47, df=75.51329508103238)

In [ ]:
stats.ttest_ind(d1['PetalWidthCm'], d3['PetalWidthCm'], equal_var = False)

In [ ]:
stats.ttest_ind(d2['PetalWidthCm'], d3['PetalWidthCm'], equal_var = False)

In [15]:
iris = iris.drop('Id', axis=1)

In [28]:
profile = ProfileReport(iris, title= 'Relatório - Pandas Profiling', html={'style':{'full_width':True}})

In [ ]:
profile

In [ ]:
profile.to_file(output_file="Relatorio01.html")